In [287]:
import re 
import string
import timestring
from sklearn.linear_model import LinearRegression, LogisticRegression
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import cross_val_score, KFold
import glob, os
import numpy as np

In [234]:
def tokenize(text):
    punc_re = '[' + '\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\_\\{\\|\\}' + ']'
    #print punc_re
    text = text.lower()
    text = re.sub('#(\S+)', r'HASHTAG_\1', text)
    text = re.sub('@(\S+)', r'MENTION_\1', text)
    text = re.sub('http\S+', 'THIS_IS_A_URL', text)
    text = re.sub(r'(.)\1\1\1+', r'\1', text)
    text = re.sub(r'[0-9]', '9', text)
    toks = []
    for tok in text.split():
        tok = re.sub(r'^(' + punc_re + '+)', r'\1 ', tok)
        #print '1',tok
        tok = re.sub(r'(' + punc_re + '+)$', r' \1', tok)
        #print '2',tok

        for subtok in tok.split():
            #print '3',subtok
            if re.search('\w', subtok):
                toks.append(subtok)
    return toks

In [235]:
DIR = 'D:/TCC/Data/Timeline/vemprarua/'
import os, io, json, codecs

os.chdir(DIR)
date_ative = []
countuser = 0
user = []

#for file in glob.glob("*.txt"):
 #   print 'file ', file, '\n'
with io.open('CleuzaBapti.txt','r',encoding="utf-8") as f, codecs.open("CleuzaBapti - Copy.txt", 'w+', encoding='utf-8') as fi:
    for line in  reversed(f.readlines()):  
        tweet= json.loads(line)
        tweet['tokened_text'] = tokenize(tweet['text'])
        fi.write(json.dumps(tweet,ensure_ascii=False,encoding='utf-8')+'\n')

        #if countuser == 100:
         #   break
        countuser+=1


In [329]:
DIR= 'C:/Users/Elaine/Documents/Data/Timeline/teste/'
os.chdir(DIR)

positive_tweets = []
easy_negative_sample = []
hard_negative_sample = []
N=5
count=0
for file in glob.glob("*.txt"):
    print 'file ', file, '\n'
    with io.open(DIR+file,'r',encoding="utf-8") as f:
        lines = f.readlines()
        i=0
        for i in xrange(len(lines)):
            tweet= json.loads(lines[i])
            if 'vempraRua' in tweet['text'] or 'vemprarua' in tweet['text'] or 'vem pra rua' in tweet['text']or 'VEMPRARUA' in tweet['text'] or 'VemPraRua' in tweet['text']:
                count= count +1
                tweet['label'] = 1
                positive_tweets.append(tweet)
                date = timestring.Date(tweet['time_tweet'])
                i=i-1
                j=i-1
                tweet_date = date
                print 'tweet date',tweet_date
                print '7 days',date-'7 day'
                while  tweet_date >= date - '7 day': 
                    tweet= json.loads(lines[i])                
                    if timestring.Date(tweet['time_tweet']) < date - '7 day' or i == 1:
                        print 'Tweet created before 7 days: \n tweet date',timestring.Date(tweet['time_tweet']) ,'<', date - '7 day'
                        break
                    else:
                        tweet['label'] = 2
                        easy_negative_sample.append(tweet)
                        i=i-1
                        tweet_date = timestring.Date(tweet['time_tweet'])
                        print i, 'i'
                        print 'date of tweet',tweet_date
                    #if i == 1:
                     #   print 'comeco do arquivo'
                     #   break

                while len(hard_negative_sample)<N:
                    tweet= json.loads(lines[j])    
                    #print tweet['time_tweet']
                    tweet['label'] = 2
                    hard_negative_sample.append(tweet)
                    j=j-1
                    if j == 1:
                        print 'comeco do arquivo j'
                        break
                i=0
                break

file  0686966667ff496.txt.txt 

tweet date 2015-08-11 00:00:00
7 days 2015-08-04 00:00:00
7 i
date of tweet 2015-08-11 00:00:00
6 i
date of tweet 2015-08-11 00:00:00
5 i
date of tweet 2015-08-11 00:00:00
4 i
date of tweet 2015-08-11 00:00:00
3 i
date of tweet 2015-08-11 00:00:00
2 i
date of tweet 2015-08-11 00:00:00
1 i
date of tweet 2015-08-11 00:00:00
Tweet created before 7 days: 
 tweet date 2015-08-11 00:00:00 < 2015-08-04 00:00:00
file  anapetilo.txt.txt 

tweet date 2015-01-21 00:00:00
7 days 2015-01-14 00:00:00
105 i
date of tweet 2015-01-21 00:00:00
104 i
date of tweet 2015-01-21 00:00:00
103 i
date of tweet 2015-01-21 00:00:00
102 i
date of tweet 2015-01-21 00:00:00
101 i
date of tweet 2015-01-21 00:00:00
100 i
date of tweet 2015-01-21 00:00:00
99 i
date of tweet 2015-01-21 00:00:00
98 i
date of tweet 2015-01-21 00:00:00
97 i
date of tweet 2015-01-21 00:00:00
96 i
date of tweet 2015-01-21 00:00:00
95 i
date of tweet 2015-01-21 00:00:00
94 i
date of tweet 2015-01-21 00:00:00
93

IndexError: list index out of range

In [330]:
count

11

In [331]:
print len(positive_tweets), positive_tweets[0]['created_at'],positive_tweets[0]['text'],positive_tweets[0]['label']
print len(easy_negative_sample)

"""for f in easy_negative_sample:
    print f['label']"""
    
print len(hard_negative_sample)
"""for f in hard_negative_sample:
    print f['label']"""

11 Tue Aug 11 10:47:52 +0000 2015 RT @16AgoEuVou: .
Bóra Brasil

#16AgoEuVou #VemPraRua16Ago #PaulistaEuVou #Tuitaço7set

@VemPraRua16Ago @PaulistaEuVou @tuitaco7set

&gt;http:… 1
7538
5


"for f in hard_negative_sample:\n    print f['label']"

In [332]:
all_tweets = easy_negative_sample+positive_tweets+hard_negative_sample
print len(all_tweets)
y = np.array([t['label'] for t in all_tweets])#savind labels of the class in y
print 'label counts=', Counter(y)

7554
label counts= Counter({2: 7543, 1: 11})


In [333]:
vectorizer = TfidfVectorizer(decode_error='ignore', ngram_range=(1, 2), max_df=1., min_df=2,
                             use_idf=True, tokenizer=tokenize, binary=False, norm='l2')
X = vectorizer.fit_transform(t['text'] for t in all_tweets)
print('Vectorized %d tweets. Found %d terms.' % (X.shape[0], X.shape[1]))
features = np.array(vectorizer.get_feature_names())

Vectorized 7554 tweets. Found 16491 terms.


In [252]:
with codecs.open("C:/Users/Elaine/"+'file_test', 'w+', encoding='utf-8') as fi:
       for f in features:
            fi.write(f)

In [334]:
def do_cv(X, y, nfolds=5):
    cv = KFold(len(y), nfolds)
    return np.mean(cross_val_score(LogisticRegression(), X, y, cv=cv))
  
print do_cv(X, y)

ValueError: The number of classes has to be greater than one.